In [1]:
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
# from tensorflow.keras.optimizers import legacy
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
dataset_dir = 'private/tomatect'
base_dir = 'private/splits'
train_pct = 0.8
validation_pct = 0.15
test_pct = 0.05
img_width = 150
img_height = 150
batch_size = 16

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [6]:
# Tentukan parameter augmentasi data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Rescaling intensitas piksel menjadi rentang [0,1]
    rotation_range=40,  # Rotasi citra hingga 40 derajat
    width_shift_range=0.2,  # Perpindahan horizontal sebesar 20% lebar citra
    height_shift_range=0.2,  # Perpindahan vertikal sebesar 20% tinggi citra
    shear_range=0.2,  # Pemotongan citra sebesar 20%
    zoom_range=0.2,  # Perbesaran citra sebesar 20%
    horizontal_flip=True,  # Membalik citra secara horizontal
    fill_mode='nearest'  # Mengisi piksel yang hilang dengan piksel terdekat
)

# Tentukan parameter augmentasi data untuk set validasi
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [8]:
# Membangun generator data pelatihan
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'  # Menggunakan klasifikasi multikelas
)

# Membangun generator data validasi
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Membangun generator data validasi
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 183 images belonging to 5 classes.
Found 30 images belonging to 5 classes.
Found 9 images belonging to 5 classes.


In [18]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [22]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size
)

Epoch 1/10


2023-06-06 20:51:03.112371: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-06 20:51:03.408337: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - ETA: 0s - loss: 1.6110 - accuracy: 0.1796

2023-06-06 20:51:12.672601: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-06 20:51:12.741650: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 11s 919ms/step - loss: 1.6110 - accuracy: 0.1796 - val_loss: 1.5818 - val_accuracy: 0.2500
Epoch 2/10
11/11 [==============================] - 9s 817ms/step - loss: 1.5946 - accuracy: 0.2156 - val_loss: 1.5569 - val_accuracy: 0.3125
Epoch 3/10
11/11 [==============================] - 9s 837ms/step - loss: 1.5932 - accuracy: 0.1976 - val_loss: 1.6059 - val_accuracy: 0.1250
Epoch 4/10
11/11 [==============================] - 9s 852ms/step - loss: 1.5761 - accuracy: 0.2395 - val_loss: 1.5512 - val_accuracy: 0.3125
Epoch 5/10
11/11 [==============================] - 9s 845ms/step - loss: 1.5169 - accuracy: 0.2335 - val_loss: 1.5459 - val_accuracy: 0.1875
Epoch 6/10
11/11 [==============================] - 9s 829ms/step - loss: 1.4748 - accuracy: 0.3952 - val_loss: 1.5511 - val_accuracy: 0.2500
Epoch 7/10
11/11 [==============================] - 9s 786ms/step - loss: 1.4528 - accuracy: 0.3892 - val_loss: 1.6060 - val_accuracy: 0.3125
Epoch 8/10
11/11

In [26]:
score = model.evaluate(test_generator, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

2023-06-06 20:53:06.237441: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Validation loss: 1.3415595293045044
Validation accuracy: 0.3333333432674408
